# Prototype topic models

This document leans on models generated by a sister Jupyter notebook: `modeling.ipynb`

In [1]:
from gensim.models import LsiModel
import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle
import gensim

from pprint import pprint

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import LsiModel

from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel
import itertools as it

import en_core_web_sm

import spacy
nlp = spacy.load('en')

/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/usr/local/lib/python3.6/site-packages/msgpack_numpy

/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationW

In [2]:
# # load the finished bag-of-words corpus from disk
# trigram_bow_corpus = MmCorpus('../data/models_data_lower/spacy_trigram_bow_corpus_all.mm') # No POS preprocessing

# # load the finished dictionary from disk
# trigram_dictionary = Dictionary.load('../data/models_data_lower/spacy_trigram_dict_all.dict') # No POS preprocessing

# load the finished bag-of-words corpus from disk
trigram_bow_corpus_POS = MmCorpus('../data/spacy_trigram_bow_corpus_all_POS.mm') # With POS preprocessing

# load the finished dictionary from disk
trigram_dictionary_POS = Dictionary.load('../data/spacy_trigram_dict_all_POS.dict') # With POS preprocessing

In [24]:
%%time
# Create LDA model

with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    # workers => sets the parallelism, and should be
    # set to your number of physical cores minus one
    lda_alpha_auto = LdaModel(trigram_bow_corpus_POS, 
                                 id2word=trigram_dictionary_POS, 
                                 num_topics=25,
                             alpha='auto', eta='auto')
    
    lda_alpha_auto.save('../data/models_data_lower/spacy_lda_model_POS_alpha_eta_auto')

CPU times: user 2min 2s, sys: 1.07 s, total: 2min 3s
Wall time: 2min 4s


In [25]:
# load the finished LDA model from disk
lda = LdaModel.load('../data/models_data_lower/spacy_lda_model_POS_alpha_eta_auto')

In [5]:
# # For LDA 25-topic Gensim model -- this version is numbered based on pyLDAvis, not Gensim -- needs to be Gensim numbering
# topic_names_vis = {1: u'Supervisor, QA, and Process Improvement',
#                2: u'DevOps',
#                3: u'Data Science',
#                4: u'UX and Design',
#                5: u'Marketing and Growth Strategy',
#                6: u'Web Development (More Frontend)',
#                7: u'Sales',
#                8: u'Technical Federal Contracting and Cybersecurity',
#                9: u'Mobile Developer',
#                10: u'Database Administrator',
#                11: u'Quality Assurance and Testing',
#                12: u'Students, Interns, CMS/Marketing, Benefits (?)',
#                13: u'Financial Risk and Cybersecurity',
#                14: u'Networks, Hardware, Linux',
#                15: u'Large Tech Corps (NVIDIA, Splunk, Twitch) (?)',
#                16: u'Big Data Engineering',
#                17: u'Health Care (Data Systems)',
#                18: u'Social Media Advertising Management',
#                19: u'Amazon Engineering/Computing/Robotics/AI (?)',
#                20: u'Fintech and Investing Analysis/Data/Engineering',
#                21: u'Defense Contracting',
#                22: u'Social Media Marketing',
#                23: u'Large Tech Corps II: Fiserv, Adove, SAP; JD chaff',
#                24: u'Flight and Space (Hardware & Software)',
#                25: u'Payments and Blockchain'}

In [26]:
# Redone for Gensim topic numbers, which differ from pyLDAvis numbers

topic_names = {1: u'(?)Large Tech Corps (NVIDIA, Splunk, Twitch)',
               2: u'Technical Federal Contracting and Cybersecurity',
               3: u'Financial Risk and Cybersecurity',
               4: u'Web Development (More Frontend)',
               5: u'Social Media Marketing',
               6: u'Fintech, Accounting, and Investing Analysis/Data',
               7: u'(?)Students, Interns, CMS/Marketing, Benefits',
               8: u'Health Care (Data Systems)',
               9: u'Database Administrator',
               10: u'Marketing and Growth Strategy',
               11: u'Quality Assurance and Testing',
               12: u'Data Science',
               13: u'Big Data Engineering',
               14: u'Sales',
               15: u'(?)Large Tech Corps Chaff: Fiserv, Adove, SAP',
               16: u'Flight and Space (Hardware & Software)',
               17: u'Networks, Hardware, Linux',
               18: u'Supervisor, QA, and Process Improvement',
               19: u'Defense Contracting',
               20: u'Social Media Advertising Management',
               21: u'UX and Design',
               22: u'(?)Amazon Engineering/Computing/Robotics/AI',
               23: u'Mobile Developer',
               24: u'DevOps',
               25: u'Payments, Finance, and Blockchain'}

In [5]:
topic_names_filepath = '../models/topic_names.pkl'

with open(topic_names_filepath, 'wb') as f:
    pickle.dump(topic_names, f)

## Visualize Topics

In [27]:
LDAvis_data_filepath = '../models/ldavis_prepared_alpha_eta_auto'

In [28]:
%%time

LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus_POS,
                                          trigram_dictionary_POS)

with open(LDAvis_data_filepath, 'wb') as f:
    pickle.dump(LDAvis_prepared, f)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


CPU times: user 1min 33s, sys: 882 ms, total: 1min 34s
Wall time: 1min 39s


In [7]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [29]:
pyLDAvis.display(LDAvis_prepared)

*Side note: export as standalone webpage or as JSON:*

In [42]:
pyLDAvis.save_html(LDAvis_prepared, '../html/lda.html')

In [150]:
pyLDAvis.save_json(LDAvis_prepared, '../html/lda.json')

## Operationalize Model
1. Preprocess an input resume
2. Compare an input resume to a topic (or variety of topics)
3. Show delta between input 

## General TODO
- Lowercase all data before preprocessing

## Find top probability job for an input document

In [9]:
# Preprocess steps for a document
import nltk
from nltk.corpus import stopwords
from nltk import RegexpTokenizer


stopwords = stopwords.words('english')
lemmatizer = nltk.WordNetLemmatizer()

def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
        yield subtree.leaves()

def normalise(word):
    word = word.lower().replace('/','').replace('-','').replace('•','')
    # word = stemmer.stem_word(word) #if we consider stemmer then results comes with stemmed word, but in this case word will not match with comment
    word = lemmatizer.lemmatize(word)
    return word

def acceptable_word(word):
    """Checks conditions for acceptable word: length, stopword. We can increase the length if we want to consider large phrase"""
    accepted = bool(2 <= len(word) <= 40
        and word.lower() not in stopwords)
    return accepted

def get_terms(tree):
    for leaf in leaves(tree):
        term = [ normalise(w) for w,t in leaf if acceptable_word(w) ]
        yield term
        
# combine functions above
def noun_phrases(text):
    tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')    
    lemmatizer = nltk.WordNetLemmatizer()
    stemmer = nltk.stem.porter.PorterStemmer()
    grammar = r"""
        NBAR:
            {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns

        NP:
            {<NBAR>}
            {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
    """
    chunker = nltk.RegexpParser(grammar)
    toks = tokenizer.tokenize(text)
    postoks = nltk.tag.pos_tag(toks)
    tree = chunker.parse(postoks)
    terms = get_terms(tree)
    bad_words = ['opportunity', 'ideal candidate', 'team', 'year', 'knowledge','experience']
    clean_terms = []
    
    for term in terms:
        term = ' '.join(term).replace('\n','').replace(',','').replace('(','')
        term = term.replace(')','')
        term = term.strip()
        if term not in bad_words:
            clean_terms.append(term)
    return clean_terms

def parse_input_docs(doc):
    return ' '.join(noun_phrases(doc))


<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence \w
<input>:34: DeprecationWarning: invalid escape sequence 

In [39]:
# %%time
# index = gensim.similarities.MatrixSimilarity(lda[trigram_bow_corpus_POS])
# index.save("simIndex_alpha_eta_auto.index")

In [44]:
# input document
with open('../data/sample_resume.txt', 'r') as infile:
    resume = infile.read()
doc = parse_input_docs(resume)

# cast to LDA space
vec_bow = trigram_dictionary_POS.doc2bow(doc.lower().split())
doc_lda = lda[vec_bow]

In [64]:
import boto3
import json
s3 = boto3.resource('s3')
BUCKET_NAME = 'tech-salary-project'

In [66]:
%%time

# Pull down original input data into memory so that the similarity queries are grounded in real data

bucket = s3.Bucket(BUCKET_NAME)
all_job_titles = []
all_job_ids = []
all_summaries = []
for o in bucket.objects.all():
    if o.key.startswith('salaries'):
        continue
    
    object = bucket.Object(o.key)
    try :
        lines = object.get()['Body'].read().decode('utf-8').splitlines()
        for line in lines:
            d = json.loads(line)
            
            title = d['title']
            jid = d['id']
            summary = d['summary']
            
            all_job_ids.append(jid)
            all_job_titles.append(title)
            all_summaries.append(summary)
    except Exception as e:
        continue

CPU times: user 13.5 s, sys: 1.56 s, total: 15 s
Wall time: 7min 22s


In [71]:
import os

In [86]:
from bs4 import BeautifulSoup
from bs4.element import Comment

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def make_text(text):
    soup = BeautifulSoup(text, "html5lib")
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip() for t in visible_texts)

In [41]:
# # Use index to find similar documents
# sims = index[vec_lda]
# sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
# # pprint(sims[:10])

# ctr = 0
# for item in sims_sorted[:5]:
#     ctr += 1
#     print('#'*20+" DOCUMENT MATCH NUMBER", ctr, '#'*20, '\n\n')
#     print(all_job_titles[item[0]]+'\nID: '+all_job_ids[item[0]],"\n\tIndex:",item[0],"\n\tSimilarity:",item[1])
#     print("Summary:\n",make_text(all_summaries[item[0]]),'\n\n')

# Compare an input to topics

In [45]:
# Use LDA to find the best topics for a given input document
sorted_doc_lda = sorted(doc_lda, key=lambda review_lda: -review_lda[1])

In [46]:
# Compare an input to the terms 
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print(u'{:20} {}'.format(u'term', u'frequency') + u'')

    for term, frequency in lda.show_topic(topic_number, topn=40):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
        
for i in range(25): # Same number as the types of jobs we scraped initially
    print("\n\nTopic %s" % str(i))
    explore_topic(topic_number=i)



Topic 0
term                 frequency
cloud                0.031
platform             0.021
big_data             0.018
aws                  0.017
infrastructure       0.011
large_scale          0.010
architecture         0.009
building             0.008
database             0.007
java                 0.007
storage              0.007
scale                0.006
performance          0.006
amazon               0.006
knowledge            0.006
problem              0.006
analytics            0.006
role                 0.005
’s                   0.005
scalable             0.005
enterprise           0.005
web                  0.005
year                 0.005
microsoft            0.005
hadoop               0.004
practice             0.004
complex              0.004
distributed_system   0.004
part                 0.004
open_source          0.004
developer            0.004
preferred_qualification 0.004
expertise            0.004
network              0.004
understanding        0.004
oracle     

## TODO
1. Do a more thorough job comparing input resumes to topics. Show the delta between a resume and a given topic. That's the primary output of the application.
2. Do the same ^^ for "other near topics"
3. Find some way to connect salary data to all of this

# Using Same Preprocessing Methods as Original Formatting

In [10]:
# Original functions from processing step, in modeling.ipynb
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    SRG: modified for a list
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    for review in filename:
        yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [31]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
# Load up the bigram and trigram models we trained earlier
bigram_model = Phrases.load('../models/spacy_bigram_model_all_PARSED_POS')
trigram_model = Phrases.load('../models/spacy_trigram_model_all_PARSED_POS')

In [32]:
trigram_dictionary = trigram_dictionary_POS
def vectorize_input(input_doc, bigram_model, trigram_model, trigram_dictionary):
    """
    (1) parse input doc with spaCy, apply text pre-proccessing steps, 
    (3) create a bag-of-words representation (4) create an LDA representation
    """
    
    # parse the review text with spaCy
    parsed_doc = nlp(input_doc)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_doc = [token.lemma_ for token in parsed_doc
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_doc = bigram_model[unigram_doc]
    trigram_doc = trigram_model[bigram_doc]
    
    # remove any remaining stopwords
    trigram_review = [term for term in trigram_doc
                      if not term in stopwords]
    
    # create a bag-of-words representation
    doc_bow = trigram_dictionary_POS.doc2bow(trigram_doc)

    # create an LDA representation
    document_lda = lda[doc_bow]
    return trigram_review, document_lda

def lda_top_topics(document_lda, topic_names, min_topic_freq=0.05):
    '''
    Print a sorted list of the top topics for a given LDA representation
    '''
    # sort with the most highly related topics first
    sorted_doc_lda = sorted(document_lda, key=lambda review_lda: -review_lda[1])
    
    for topic_number, freq in sorted_doc_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print('*'*56) 
        print('{:50} {:.3f}'.format(topic_names[topic_number+1],
                                round(freq, 3)))
        print('*'*56)
        for term, term_freq in lda.show_topic(topic_number, topn=10):
            print(u'{:20} {:.3f}'.format(term, round(term_freq, 3)))
        print('\n\n')

def top_match_items(document_lda, topic_names, num_terms=100):
    '''
    Print a sorted list of the top topics for a given LDA representation
    '''
    # sort with the most highly related topics first
    sorted_doc_lda = sorted(document_lda, key=lambda review_lda: -review_lda[1])

    topic_number, freq = sorted_doc_lda[0][0], sorted_doc_lda[0][1]
    print('*'*56)
    print('{:50} {:.3f}'.format(topic_names[topic_number+1],
                            round(freq, 3)))
    print('*'*56)
    for term, term_freq in lda.show_topic(topic_number, topn=num_terms):
        print(u'{:20} {:.3f}'.format(term, round(term_freq, 3)))


def top_match_list(document_lda, topic_names, num_terms=100):
    # Take the above results and just save to a list of the top 500 terms in the topic
    sorted_doc_lda = sorted(document_lda, key=lambda review_lda: -review_lda[1])
    topic_number, freq = sorted_doc_lda[0][0], sorted_doc_lda[0][1]
    print('Highest probability topic:', topic_names[topic_number+1], 
                                         '\t', round(freq, 3))
    top_topic_skills = []
    for term, term_freq in lda.show_topic(topic_number, topn=num_terms):
        top_topic_skills.append(term)
    return top_topic_skills

def common_skills(top_topic_skills, user_skills):
    return [item for item in top_topic_skills if item in user_skills]

def non_common_skills(top_topic_skills, user_skills):
    return [item for item in top_topic_skills if item not in user_skills]

In [ ]:
with open('../data/sample_resume.txt', 'r') as infile:
    sample1 = infile.read()
with open('../data/sample_ds_resume2.txt', 'r') as infile:
    sample2 = infile.read()

In [33]:
user_skills, my_lda = vectorize_input(sample1, bigram_model, trigram_model, trigram_dictionary)

# top_items = top_match_items(my_lda, topic_names)
# print(top_items)

skills_list = top_match_list(my_lda, topic_names, num_terms=500)

print("Top 20 skills user has in common with topic:")
pprint(common_skills(skills_list, user_skills)[:20])


print("\n\nTop 20 skills user DOES NOT have in common with topic:")
pprint(non_common_skills(skills_list, user_skills)[:20])


Highest probability topic: Technical Federal Contracting and Cybersecurity 	 0.207
Top 20 skills user has in common with topic:
['employee',
 '’s',
 'growth',
 'financial',
 'web',
 'mission',
 'tech',
 'building',
 'big',
 'organization',
 'member',
 'job',
 'feature',
 'education',
 'language',
 'personal',
 'test',
 'information',
 'need',
 'week']


Top 20 skills user DOES NOT have in common with topic:
['people',
 'world',
 'benefit',
 'life',
 'time',
 'great',
 'way',
 'culture',
 'platform',
 'part',
 'office',
 'code',
 'problem',
 'health',
 'fun',
 'challenge',
 'career',
 'innovative',
 'community',
 'individual']


/usr/local/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


# Assess relevance vs. salience for topic terms

Look at this code: http://pyldavis.readthedocs.io/en/latest/_modules/pyLDAvis/_prepare.html 

In [ ]:
def _topic_info(topic_term_dists, topic_proportion, term_frequency, term_topic_freq, vocab, lambda_step, R, n_jobs):
    # marginal distribution over terms (width of blue bars)
    term_proportion = term_frequency / term_frequency.sum()

    # compute the distinctiveness and saliency of the terms:
    # this determines the R terms that are displayed when no topic is selected
    topic_given_term = topic_term_dists / topic_term_dists.sum()
    kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
    distinctiveness = kernel.sum()
    saliency = term_proportion * distinctiveness

    # Order the terms for the "default" view by decreasing saliency:
    default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \
                                      'Freq': term_frequency, 'Total': term_frequency, \
                                      'Category': 'Default'}). \
      sort_values(by='saliency', ascending=False).head(R).drop('saliency', 1)
    # Rounding Freq and Total to integer values to match LDAvis code:
    default_term_info['Freq'] = np.floor(default_term_info['Freq'])
    default_term_info['Total'] = np.floor(default_term_info['Total'])
    ranks = np.arange(R, 0, -1)
    default_term_info['logprob'] = default_term_info['loglift'] = ranks

    ## compute relevance and top terms for each topic
    log_lift = np.log(topic_term_dists / term_proportion)
    log_ttd = np.log(topic_term_dists)
    lambda_seq = np.arange(0, 1 + lambda_step, lambda_step)

    def topic_top_term_df(tup):
      new_topic_id, (original_topic_id, topic_terms) = tup
      term_ix = topic_terms.unique()
      return pd.DataFrame({'Term': vocab[term_ix], \
                           'Freq': term_topic_freq.loc[original_topic_id, term_ix], \
                           'Total': term_frequency[term_ix], \
                           'logprob': log_ttd.loc[original_topic_id, term_ix].round(4), \
                           'loglift': log_lift.loc[original_topic_id, term_ix].round(4), \
                           'Category': 'Topic%d' % new_topic_id})

    top_terms = pd.concat(Parallel(n_jobs=n_jobs)(delayed(_find_relevance_chunks)(log_ttd, log_lift, R, ls) \
                                                 for ls in _job_chunks(lambda_seq, n_jobs)))
    topic_dfs = map(topic_top_term_df, enumerate(top_terms.T.iterrows(), 1))
    return pd.concat([default_term_info] + list(topic_dfs))

In [ ]:
def _find_relevance(log_ttd, log_lift, R, lambda_ = 0.5):
    relevance = lambda_ * log_ttd + (1 - lambda_) * log_lift
    return relevance.T.apply(lambda s: s.sort_values(ascending=False).index).head(R)

## Refinement
- Matching using overall term frequency rather than topic-specific term frequency


### Next Steps
- Fact that we're matching resumes to JDs? Different doc types?
- Hyperparameter tuning with model? 
    - Number of topics?
    - Number iterations?
- Increase volume of input data
- Look at whether there is a precipitous drop from the highest-probability topic to number 2, 3, etc.

In [88]:
# def explore_topic(topic_number, topn=25):
#     """
#     accept a user-supplied topic number and
#     print out a formatted list of the top terms
#     """
        
# #     print(u'{:20} {}'.format(u'term', u'frequency') + u'')

#     for term, frequency in lda.show_topic(topic_number, topn=100):
#         print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
        
# for i in range(25): # Same number as the types of jobs we scraped initially
#     print("\n\n%s." % str(i+1), topic_names_vis[i+1])
#     explore_topic(topic_number=i)

# TODO
- Add details for each topic (create a list of top ~500 terms for each topic; return "in common" terms and "not in common terms"
- Attempt this with a model that has many more topics, and see how the results look. Maybe try 50? Requires manual labelling (again)

# Questions
- How do I return Gensim topics with high levels of salience vs relevance?